In [100]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [101]:
dataframe = pd.read_excel("Gem comparison products copy.xlsx")
dataframe = dataframe.loc[:, ['URL', 'CURRENT PRICE']]
p_index = 1

In [102]:
dataframe.head()

,URL,CURRENT PRICE
0,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,131
1,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,138
2,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,148
3,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,148
4,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,148


In [103]:
product_url = dataframe["URL"][p_index]
print(product_url)

https://mkp.gem.gov.in/mcb-miniature-circuit-breakers-c-operation-per-iec/20a-sp-c-mcb/p-5116877-295066335-cat.html#variant_id=5116877-295066335


In [104]:
# product_url = "https://mkp.gem.gov.in/electric-ceiling-type-fan-v2-per-374/bajaj-bldc-28-w/p-5116877-29311430059-cat.html"

In [105]:
product_url = "https://mkp.gem.gov.in/mcb-miniature-circuit-breakers-c-operation-per-iec/20a-sp-c-mcb/p-5116877-295066335-cat.html#variant_id=5116877-295066335"

In [106]:
competitors_url = re.sub(".html.*", "/all_sellers.html", product_url)

In [107]:
competitors_url_response = requests.get(competitors_url)

In [108]:
competitors_url_soup = BeautifulSoup(competitors_url_response.content, 'html.parser')

In [109]:
competitors_prices_table = competitors_url_soup.find('div', attrs={'id': 'sellers-table-wrap'})
competitors_prices_table = competitors_prices_table.find('table')
for div in competitors_prices_table.find_all("div", {'class':'seller-info-table'}):
    div.decompose()

In [110]:
table_rows = competitors_prices_table.find_all('tr')

res = []
for i, tr in enumerate(table_rows):
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Sellers", "Offer Price", "Delivery Locations", "Quantity based Discount", "Quantity Available", "Min Quantity / Consignee", "Offer Product As", "Country of Origin"])
df = df[["Offer Price", "Delivery Locations", "Quantity based Discount", "Min Quantity / Consignee"]]

In [111]:
df

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee
0,₹235.00,State(s) - JAMMU & KASHMIR,,1
1,₹260.00,All India,,2
2,₹140.00,State(s) - PUNJAB,,12
3,₹263.00,State(s) - MAHARASHTRA,,10
4,₹263.00,"District(s) - JODHPUR, RAJASTHAN",,50
5,₹263.00,State(s) - MAHARASHTRA,,10
6,₹118.80,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,200
7,₹117.88,All India,,20
8,₹138.98,All India,,40
9,₹255.00,"State(s) - WEST BENGAL, NAGALAND",219 to 220 units = ₹186.99,220


In [125]:
delivery_locations_regex = "|".join(["TAMIL NADU", "All India"])
competitors_details = df[
    df["Delivery Locations"].str.contains(delivery_locations_regex)
]

In [126]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee
1,₹260.00,All India,,2
6,₹118.80,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,200
7,₹117.88,All India,,20
8,₹138.98,All India,,40
10,₹140.00,All India,,100
11,₹255.00,All India,849 to 850 units = ₹189.00,220
12,₹263.70,"State(s) - KARNATAKA, TAMIL NADU, MAHARASHTRA,...",,10
15,₹260.00,All India,,40
16,₹263.50,"State(s) - KARNATAKA, TAMIL NADU, ANDHRA PRADE...",97 to 98 units = ₹24.08,1
17,₹118.00,All India,2 to 500 units = ₹0.11,25


In [127]:
competitors_details["Offer Price"] = (
    competitors_details["Offer Price"].str.slice(1)
)
competitors_details["Offer Price"] = (
    competitors_details["Offer Price"].str.replace(",", "").astype(float)
)

C:\Users\nikun\AppData\Local\Temp\ipykernel_30736\717331536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitors_details["Offer Price"] = (
C:\Users\nikun\AppData\Local\Temp\ipykernel_30736\717331536.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitors_details["Offer Price"] = (


In [128]:
def determine_final_price(row):
    discount_info = row["Quantity based Discount"]

    if discount_info == "":
        return row["Offer Price"]

    discounts = discount_info.split("\n")
    final_price = row["Offer Price"]

    for discount_entry in discounts:
        discount_entry = discount_entry.strip()
        parts = discount_entry.split(" ")
        discount = float(parts[5][1:].replace(",", ""))
        price = row["Offer Price"] - discount
        min_quantity = int(parts[0])

        if price * min_quantity <= 25000 and price < final_price:
            final_price = price

    return final_price

In [130]:
competitors_details["Final Price"] = competitors_details.apply(
    determine_final_price, axis=1
)

C:\Users\nikun\AppData\Local\Temp\ipykernel_30736\3674593290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitors_details["Final Price"] = competitors_details.apply(


In [131]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee,Final Price
1,260.00,All India,,2,260.00
6,118.80,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,200,118.80
7,117.88,All India,,20,117.88
8,138.98,All India,,40,138.98
10,140.00,All India,,100,140.00
11,255.00,All India,849 to 850 units = ₹189.00,220,255.00
12,263.70,"State(s) - KARNATAKA, TAMIL NADU, MAHARASHTRA,...",,10,263.70
15,260.00,All India,,40,260.00
16,263.50,"State(s) - KARNATAKA, TAMIL NADU, ANDHRA PRADE...",97 to 98 units = ₹24.08,1,239.42
17,118.00,All India,2 to 500 units = ₹0.11,25,117.89


In [132]:
# Drop rows where Min Quantity * Offer Price exceeds 25000
competitors_details = competitors_details[
    competitors_details["Min Quantity / Consignee"].astype(int) * competitors_details["Offer Price"]
    <= 25000
]

In [133]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee,Final Price
1,260.00,All India,,2,260.00
6,118.80,"State(s) - KARNATAKA, TAMIL NADU, JAMMU & KASH...",,200,118.80
7,117.88,All India,,20,117.88
8,138.98,All India,,40,138.98
10,140.00,All India,,100,140.00
12,263.70,"State(s) - KARNATAKA, TAMIL NADU, MAHARASHTRA,...",,10,263.70
15,260.00,All India,,40,260.00
16,263.50,"State(s) - KARNATAKA, TAMIL NADU, ANDHRA PRADE...",97 to 98 units = ₹24.08,1,239.42
17,118.00,All India,2 to 500 units = ₹0.11,25,117.89


In [119]:
# for index, row in filtered_df.iterrows():
#     if row["Quantity based Discount"] != "":
#         discount_string = row["Quantity based Discount"].split(" ")
#         discount = float("".join(discount_string[5][1:].split(",")))
#         # min_quantity = int(discount_string[0])
#         # max_quantity = int(discount_string[2])
#         new_offer_price = row["Offer Price"] - discount
#         # temp = 25000/new_offer_price
#         # print("Values:", row["Offer Price"], "|", discount, "|", new_offer_price, "|", temp, "|", min_quantity, "|", max_quantity)
#         # if temp >= min_quantity and temp <= max_quantity:
#         # if new_offer_price * row["Min Quantity / Consignee"]
#         row["Offer Price"] = new_offer_price
#         print("Price changed:", index)
#     if int(row["Min Quantity / Consignee"]) * row["Offer Price"] > 25000:
#         filtered_df.drop(index)
#         print("Dropped:", index)

In [134]:
competitors_min_price = float(competitors_details['Offer Price'].min())

In [135]:
competitors_min_price

117.88

In [122]:
product_price = dataframe["CURRENT PRICE"][p_index]

In [123]:
product_price

138

In [124]:
product_price > competitors_min_price

True